# PCAP Lab - 5: CUDA

<h4>Parthivi Choubey</h4>

*180905456*

CSE - B - 6th sem

Roll. no.: 60

# CUDA installation and update

In [ ]:
%%bash
apt-get --purge remove cuda nvidia* libnvidia-*
dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
apt-get remove cuda-*
apt autoremove
apt-get update

In [ ]:
%%bash
wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
apt-get update
apt-get install cuda-9.2

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# Question 1
Write and execute a program in CUDA to add two vectors of length N to meet the following requirements using 3 different kernels  

1. block size as N  
2. N threads within a block
3. Keep the number of threads per block as 256 (constant) and vary the number of blocks to handle N elements.  

In [40]:
%%cu
// Q1
#include <cuda.h>
#include <stdlib.h>
#include <stdio.h>

__global__ void vecAddKernel(int* a, int* b, int* c, int n)
{
    int id = threadIdx.x + blockDim.x*blockIdx.x;
    if(id<n)
        c[id]=a[id]+b[id];
}

void vecAdd(int* a, int* b, int* c, int n)
{
    int *d_a, *d_b, *d_c; // Device copies of the variables
    int size=n*sizeof(int);
    
    // Allocate memories to device copies of the objects
    cudaMalloc((void**)&d_a, size);
    cudaMalloc((void**)&d_b, size);
    cudaMalloc((void**)&d_c, size);
 
    // Copy inputs to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    printf("A: ");
    for (int i = 0; i < n; i++)
        printf("%d ", a[i]);
    printf("\n");
 
    printf("B: ");
    for (int i = 0; i < n; i++)
        printf("%d ", b[i]);
    printf("\n\n");

    vecAddKernel<<<n,1>>>(d_a, d_b, d_c, n); // Launch kernel onto the device
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost); // Copy the result back to the host
    printf("a+b using %d blocks with 1 thread each: ",n);
    for (int i = 0; i < n ; i++)
        printf("%d ", c[i]);
    printf("\n");

    vecAddKernel<<<1,n>>>(d_a, d_b, d_c, n); // Launch kernel onto the device
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost); // Copy the result back to the host
    printf("a+b using %d threads in 1 block: ",n);
    for (int i = 0; i < n ; i++)
        printf("%d ", c[i]);
    printf("\n");

    vecAddKernel<<<ceil(n/256.0),256>>>(d_a, d_b, d_c, n); // Launch kernel onto the device
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost); // Copy the result back to the host
    printf("a+b using variable blocks and 256 threads: ");
    for (int i = 0; i < n ; i++)
        printf("%d ", c[i]);
    printf("\n");
 
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
}

int main()
{
    int n = 272;
    int h_A[n], h_B[n], h_C[n]; // Host copies of the variables
 
    for (int i = 0; i < n; i++)
    {
        h_A[i] = i;
        h_B[i] = i*4;
    }

    vecAdd(h_A, h_B, h_C, n);

    return 0;
}

A: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 
B: 0 4 8 12 16 20 

# Question 2
Write  and execute a CUDA program to read an array of N integer values. Sort the array in parallel using parallel selection sort and store the result in another array.  

In [42]:
%%cu
// Q2
#include<stdio.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__global__ void selection_sort_parallel(int* a, int* b, int n)
{
	int tid=threadIdx.x;
  int pos=0;
  if(tid<n){
      for(int j=0;j<n;j++){
          if(a[j]<a[tid]||(a[j]==a[tid] && j<tid)){
              pos=pos+1;
          }
      }
      b[pos]=a[tid];
  }
}

int main()
{
  int n = 10;
	int b[n];
	int *d_a,*d_b; // Device copies of the variables
  int size=sizeof(int) * n;
  int a[10]={7,69,8,2,420,173,55,8,93,16};
  printf("Unsorted array ");
  for(int i=0;i<n;i++)
      printf("%d ",a[i]);
  printf("\n");
 
  // Allocate memories to device copies of the objects
	cudaMalloc((void **)&d_a,size);
  cudaMalloc((void **)&d_b,size);
 
	cudaMemcpy(d_a,&a,size,cudaMemcpyHostToDevice);
  cudaMemcpy(d_b,&b,size,cudaMemcpyHostToDevice);
  selection_sort_parallel<<<1,n>>>(d_a,d_b,n);
  cudaMemcpy(&b,d_b,size,cudaMemcpyDeviceToHost);
  printf("Sorted array ");
  for(int i=0;i<n;i++)
      printf("%d ",b[i]);
 
  cudaFree(d_a);
  cudaFree(d_b);
}

Unsorted array 7 69 8 2 420 173 55 8 93 16 
Sorted array 2 7 8 8 16 55 69 93 173 420 


# Question 3
Write a execute a CUDA program to read an integer array of size N. Sort this array using odd-even transposition sorting. Use 2 kernels.

In [41]:
%%cu
// Q3
#include <stdio.h>
#include <stdlib.h>

__global__ void oddeven(int *a, int n)
{
    int temp, id = threadIdx.x + blockDim.x * blockIdx.x;
    if(id>n) return;
    if(id%2==1 && id+1<n)
    {
        if(a[id]>a[id+1])
        {
            temp = a[id];
            a[id] = a[id+1];
            a[id+1] = temp;
        }
    }
}
__global__ void evenodd(int *a, int n)
{
    int temp, id = threadIdx.x + blockDim.x * blockIdx.x;
    if(id>n) return;
    if(id%2==0 && id+1<n)
    {
        if(a[id]>a[id+1])
        {
            temp = a[id];
            a[id] = a[id+1];
            a[id+1] = temp;
        }
    }
}
void hostfunc(int *a, int n)
{
    int size = n * sizeof(int);
    int *d_a; // Device copy of the variable
    cudaMalloc((void**)&d_a,size); // Allocate memory to device copy of the object
    cudaMemcpy(d_a,a,size,cudaMemcpyHostToDevice); // Copy input to device
 
    for(int i=0;i<n/2;i++)
    {
        // Launch kernel onto the device
        oddeven<<<1,n>>>(d_a,n);
        evenodd<<<1,n>>>(d_a,n);
    }
    cudaMemcpy(a,d_a,size,cudaMemcpyDeviceToHost); // Copy the result back to the host
    cudaFree(d_a);
}
int main()
{
    int n=10;
    int a[10]={7,69,8,2,420,173,55,8,93,16}; // Host copy of the variable
    printf("Unsorted array ");
    for(int i=0;i<n;i++)
      printf("%d ",a[i]);
    printf("\n");
    hostfunc(a,n);
    printf("Sorted array ");
    for(int i=0;i<n;i++)
        printf("%d ",a[i]);
    printf("\n");
}

Unsorted array 7 69 8 2 420 173 55 8 93 16 
Sorted array 2 7 8 8 16 55 69 93 173 420 

